In [45]:
import numpy as np
import pandas as pd
import os

In [46]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

In [47]:
CSV_PATH = os.path.join(os.getcwd(), '..\\final.csv')
ug = pd.read_csv(CSV_PATH)
ug.head()

,ID,.text:,.Pav:,.idata:,.data:,.bss:,.rdata:,.edata:,.rsrc:,.tls:,...,esi,eax,ebx,ecx,edi,ebp,esp,eip,Class,size
0,00000fa1585e99fcb5e8728b96f173ff61b08fc152e2f5...,55216,0,0,53660,0,0,0,26554,0,...,12360,28050,7434,9660,11738,10124,4018,0,SmokeLoader,4.970940
1,000285eadc4e6a68f32343409159c1b44aed8a0bc1c0cd...,108948,0,0,35594,0,33524,0,17770,0,...,17483,43039,9501,16834,13807,16826,8436,0,Loki,6.326924
2,00037a0cc29f3c99e88aeb57af189e291c6fe38b8b2527...,311819,0,0,0,0,0,0,64656,0,...,30109,84735,16436,15685,44207,11887,20710,0,njrat,13.333300
3,0004033aedd01d2928c1c31abb57633f1c493a213eae73...,800645,0,0,0,0,0,0,19250,0,...,78108,202422,43124,46524,74276,29626,19958,0,Loki,30.803483
4,000410685bd62172ae00cafa761c1420f219d323deade1...,106442,0,0,52749,0,0,0,22424,0,...,10939,37113,6741,17573,11825,24366,8272,0,Loki,6.981380


In [48]:
null_op = ug.columns[~ug.any()].tolist()
ug.drop(null_op, axis=1, inplace = True)

In [49]:
ug.drop('ID', axis = 1, inplace = True)

In [50]:
X = ug.drop("Class", axis = 1)
y = ug["Class"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 36)

In [51]:
scaler = StandardScaler()

In [52]:
clf_ranfo = RandomForestClassifier(max_depth=40, random_state=0)

In [53]:
pipe = Pipeline([('scaler', scaler), ('ranfo', clf_ranfo)])

In [54]:
pipe.fit(X_train.values, y_train.values)

Pipeline(steps=[('scaler', StandardScaler()),
                ('ranfo',
                 RandomForestClassifier(max_depth=40, random_state=0))])

In [55]:
def get_results(model, X_train, X_test):
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    print("\t"*2,"*"*8, "RESULTS FOR TRAIN SET", "*"*8)
    print(classification_report(y_train, y_pred_train))
    print("\n")
    print("\t"*2, "*"*8, "RESULTS FOR TEST SET", "*"*8)
    print(classification_report(y_test, y_pred_test))

In [65]:
get_results(pipe, X_train.values, X_test.values)

		 ******** RESULTS FOR TRAIN SET ********
                precision    recall  f1-score   support

        Amadey       1.00      1.00      1.00      2175
          Loki       1.00      0.88      0.93      6917
   SmokeLoader       1.00      0.61      0.76      6053
SnakeKeyLogger       0.65      1.00      0.79      6101
         njrat       1.00      0.98      0.99      3045
 non-malicious       1.00      0.98      0.99       768

      accuracy                           0.87     25059
     macro avg       0.94      0.91      0.91     25059
  weighted avg       0.91      0.87      0.87     25059



		 ******** RESULTS FOR TEST SET ********
                precision    recall  f1-score   support

        Amadey       0.95      0.89      0.92       528
          Loki       0.91      0.75      0.82      1769
   SmokeLoader       0.94      0.56      0.70      1526
SnakeKeyLogger       0.57      0.94      0.71      1510
         njrat       0.93      0.87      0.90       718
 non-maliciou

In [57]:
from joblib import dump, load

In [58]:
OUTPUT_FILE = 'model_ranfo_v1.joblib'

In [59]:
dump(pipe, OUTPUT_FILE) 

['model_ranfo_v1.joblib']

In [60]:
model = load(OUTPUT_FILE) 

In [61]:
ran = np.random.randint(0, 100)
eg_X = X_test.iloc[ran]
eg_y = y_test.iloc[ran]

In [62]:
model.predict([eg_X])

array(['Loki'], dtype=object)

In [63]:
eg_y

'Loki'

In [69]:
X_train.shape

(25059, 41)

In [71]:
print([x+11 for x in range(41)])

[11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51]


In [72]:
"""
    example code to load model and make prediction
    input shape: 41
"""
from joblib import load
model = load(OUTPUT_FILE)
model.predict([[11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51]])

array(['non-malicious'], dtype=object)

In [ ]:
features = ['.text:', '.idata:', '.data:', '.bss:', '.rdata:', '.edata:', '.rsrc:',
       '.tls:', '.reloc:', 'jmp', 'mov', 'retf', 'push', 'pop', 'xor', 'nop',
       'sub', 'inc', 'dec', 'add', 'imul', 'xchg', 'or', 'shr', 'cmp', 'call',
       'shl', 'ror', 'rol', 'jz', 'lea', 'movzx', 'edx', 'esi', 'eax', 'ebx',
       'ecx', 'edi', 'ebp', 'esp', 'size']